In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [2]:
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 10.113 sec


In [26]:
# SQLAlchemy connection
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn = engine.connect()

In [3]:
sql = """
            SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
                "Screw Part Number" AS Screw, "Band Feed from Band data",
                "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
                "CameraInspectionRequired", "ScrDrvChk"
            FROM tbl8Tridon 
"""

In [4]:
start = timer()
cursor.execute(sql)
result = cursor.fetchall()

In [47]:
data_type_dict = {'Feed' : float}
df = pd.DataFrame.from_records(result)
df.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk']
df = df[df.Feed != 'N/A']
df = df.dropna()
df = df[1:]
df.fillna({'DiaMax' : 0.0}, inplace=True)
df.fillna({'DiaMin' : 0.0}, inplace=True)
df = df.astype(data_type_dict)
df = df.convert_dtypes()
df.dtypes
# Display Duplicated Part Numbers
# ids = df['PartNumber']
# df[ids.isin(ids[ids.duplicated()])].sort_values('PartNumber')


PartNumber       string[python]
Band             string[python]
Housing          string[python]
Screw            string[python]
Feed                    Float64
DiaMax                  Float64
DiaMin                  Float64
HexSz            string[python]
BandThickness           Float64
BandWidth               Float64
CamInspect       string[python]
ScrDrvChk        string[python]
dtype: object

In [44]:
df.to_sql('parts_clamps', conn, schema='production', if_exists='replace', index=False)

169